In [1]:
%matplotlib inline
import torch
import torch.nn as nn
import numpy as np
import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l

In [2]:
def dropout(X,drop_prob):
    X = X.float()
    assert 0 <= drop_prob <= 1
    keep_prob = 1 - drop_prob
    if keep_prob == 0:
        return torch.zeros_like(X)
    mask = (torch.rand(X.shape)<keep_prob).float()
    return mask*X/keep_prob

In [3]:
X = torch.arange(16).view(2,8)
dropout(X,0.5)

tensor([[ 0.,  2.,  4.,  0.,  8.,  0., 12.,  0.],
        [16., 18., 20., 22., 24.,  0.,  0.,  0.]])

In [4]:
dropout(X,1)

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

定义模型

In [6]:
num_inputs,num_outputs,num_hiddens1,num_hiddens2 = 784,10,256,256
W1 = torch.tensor(np.random.normal(0,0.01,size=(num_inputs,num_hiddens1)),dtype=torch.float,requires_grad=True)
b1 = torch.zeros(num_hiddens1,requires_grad=True)
W2 = torch.tensor(np.random.normal(0,0.01,size=(num_hiddens1,num_hiddens2)),dtype=torch.float,requires_grad=True)
b2 = torch.zeros(num_hiddens2,requires_grad=True)
W3 = torch.tensor(np.random.normal(0,0.01,size=(num_hiddens2,num_outputs)),dtype=torch.float,requires_grad=True)
b3 = torch.zeros(num_outputs,requires_grad=True)
params = [W1,b1,W2,b2,W3,b3]

In [10]:
drop_prob1,drop_prob2 = 0.2,0.5
def net(X,is_training=True):
    X = X.view(-1,num_inputs)
    H1 = (torch.matmul(X,W1)+b1).relu()
    if is_training:
        H1 = dropout(H1,drop_prob1)
    H2 = (torch.matmul(H1,W2)+b2).relu()
    if is_training:
        H2 = dropout(H2,drop_prob2)
    return torch.matmul(H2,W3) + b3

In [11]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        if isinstance(net, torch.nn.Module):
            net.eval() # 评估模式, 这会关闭dropout
            acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
            net.train() # 改回训练模式
        else: # 自定义的模型
            if('is_training' in net.__code__.co_varnames): # 如果有is_training这个参数
                # 将is_training设置成False
                acc_sum += (net(X, is_training=False).argmax(dim=1) == y).float().sum().item() 
            else:
                acc_sum += (net(X).argmax(dim=1) == y).float().sum().item() 
        n += y.shape[0]
    return acc_sum / n

In [12]:
num_epochs, lr, batch_size = 5, 100.0, 256
loss = torch.nn.CrossEntropyLoss()
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, params, lr)

epoch 1, loss 0.0048, train acc 0.520, test acc 0.662
epoch 2, loss 0.0024, train acc 0.778, test acc 0.785
epoch 3, loss 0.0020, train acc 0.818, test acc 0.830
epoch 4, loss 0.0018, train acc 0.835, test acc 0.824
epoch 5, loss 0.0017, train acc 0.846, test acc 0.844


In [13]:
net = nn.Sequential(
    d2l.FlattenLayer(),
    nn.Linear(num_inputs,num_hiddens1),
    nn.ReLU(),
    nn.Dropout(drop_prob1),
    nn.Linear(num_hiddens1,num_hiddens2),
    nn.ReLU(),
    nn.Dropout(drop_prob2),
    nn.Linear(num_hiddens2,num_outputs)
                   )

In [14]:
for param in net.parameters():
    nn.init.normal_(param,mean=0,std=0.01)

In [15]:
optimizer = torch.optim.SGD(net.parameters(),lr=0.5)
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None, None, optimizer)

epoch 1, loss 0.0047, train acc 0.531, test acc 0.716
epoch 2, loss 0.0023, train acc 0.781, test acc 0.736
epoch 3, loss 0.0020, train acc 0.818, test acc 0.811
epoch 4, loss 0.0018, train acc 0.837, test acc 0.788
epoch 5, loss 0.0016, train acc 0.848, test acc 0.834
